<a href="https://colab.research.google.com/github/jdbcode/G4G19/blob/master/time-series-visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api
import ee
import numpy as np
import pandas as pd
import altair as alt

ee.Authenticate()
ee.Initialize()

     |████████████████████████████████| 153kB 4.9MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.197-cp36-none-any.whl size=173332 sha256=0b58e71062b16cdf3a299ac67c55a532bce84b23facaeab4d64d3525b7186874
  Stored in directory: /root/.cache/pip/wheels/48/4c/32/2d2789871571a43a82315b8f1dd89447f93092050f13751810
Successfully built earthengine-api
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verificat

In [0]:
pdsi = ee.ImageCollection("IDAHO_EPSCOR/PDSI")
sn = ee.FeatureCollection("EPA/Ecoregions/2013/L3") \
  .filter(ee.Filter.eq('na_l3name', 'Sierra Nevada'))

In [0]:
def reducePDSI(img):
  eeDate = img.date()
  year = eeDate.get('year')
  month = eeDate.getRelative('month', 'year')
  doy = eeDate.getRelative('day', 'year')
  date = eeDate.format('YYYY-MM-dd')
  
  mean = img.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = sn,
    scale = 5000,
    crs = 'EPSG:5070',
    bestEffort = True,
    maxPixels = 1e14,
    tileScale = 4
  )
  
  return(ee.Feature(None, mean)
    .set({
      'DOY': doy,
      'Month': month,
      'Year': year,
      'Date': date,
      'system:time_start': img.get('system:time_start')
    })
  )

In [0]:
snPdsiCol = pdsi.map(reducePDSI) \
  .filter(ee.Filter.notNull(['pdsi']));


snPdsiDict = snPdsiCol.reduceColumns(
  reducer = ee.Reducer.toList().repeat(5),
  selectors = ['Year', 'Month', 'DOY', 'Date', 'pdsi']
)

snPdsiList = ee.List(snPdsiDict.get('list'))

In [5]:
snPdsiList = snPdsiList.getInfo()
print(snPdsiList)

[[1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 19

In [6]:
print('n variables:', len(snPdsiList))
print('n observations:', len(snPdsiList[0]))

n variables: 5
n observations: 1462


1. Conver the list to a Pandas dataframe.
2. Print the shape of the new dataframe. `shape` returns: (n rows, n columns)

In [7]:
snPdsiDf = pd.DataFrame(snPdsiList)
print(snPdsiDf.shape)

(5, 1462)


Transpose the dataframe and check the shape again. It should now have 4 columns and 1462 rows.

In [8]:
snPdsiDf = snPdsiDf.transpose()
print(snPdsiDf.shape)

(1462, 5)


Let's take a look at the first 10 rows of the dataframe.

In [9]:
snPdsiDf.head(10).style.hide_index()

0,1,2,3,4
1979,1,31,1979-02-01,0.0423335
1979,1,40,1979-02-10,0.0321808
1979,1,50,1979-02-20,0.0170773
1979,2,59,1979-03-01,0.0846393
1979,2,68,1979-03-10,0.0931233
1979,2,78,1979-03-20,0.0377165
1979,3,90,1979-04-01,0.114435
1979,3,99,1979-04-10,0.22713
1979,3,109,1979-04-20,0.163747
1979,4,120,1979-05-01,0.0390553


We're missing column names, let's add some.

In [10]:
snPdsiDf.columns = ['Year', 'Month', 'DOY', 'Date', 'PDSI']
snPdsiDf.head(10).style.hide_index()

Year,Month,DOY,Date,PDSI
1979,1,31,1979-02-01,0.0423335
1979,1,40,1979-02-10,0.0321808
1979,1,50,1979-02-20,0.0170773
1979,2,59,1979-03-01,0.0846393
1979,2,68,1979-03-10,0.0931233
1979,2,78,1979-03-20,0.0377165
1979,3,90,1979-04-01,0.114435
1979,3,99,1979-04-10,0.22713
1979,3,109,1979-04-20,0.163747
1979,4,120,1979-05-01,0.0390553


Now check the datatype of each column.

In [11]:
snPdsiDf.dtypes

Year     object
Month    object
DOY      object
Date     object
PDSI     object
dtype: object

Set the datatypes.

In [0]:
snPdsiDf = snPdsiDf.astype({'Year': int, 'Month': int, 'DOY': int, 'Date': str, 'PDSI': float})
snPdsiDf.dtypes
snPdsiDf['Month'] = snPdsiDf['Month'] + 1

Now let's make some charts - how about a heat map

In [13]:
alt.Chart(
  snPdsiDf,
  title="PDSI Heatmap"
).mark_rect().encode(
  x='Year:O',
  y='Month:O',
  color=alt.Color('PDSI:Q', scale=alt.Scale(scheme="redblue", domain=(-5, 5))),
  tooltip=[
    alt.Tooltip('Year:O', title='Year'),
    alt.Tooltip('mean(Month):O', title='Month'),
    alt.Tooltip('PDSI:Q', title='PDSI')
  ]
).properties(width=600)

alt.Chart(...)

In [14]:
alt.Chart(
  snPdsiDf,
  title="PDSI Barchart"
).mark_bar(size=1).encode(
  x='Date:T',
  y='PDSI:Q',
  color=alt.Color('PDSI:Q', scale=alt.Scale(scheme="redblue", domain=(-5, 5))),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('PDSI:Q', title='PDSI')
  ]
).properties(width=600)

alt.Chart(...)

## Landsat NBR time series

### MODIS NDVI for phenology



In [0]:
def reduceNdvi(img):
  eeDate = img.date()
  year = eeDate.get('year')
  doy = eeDate.getRelative('day', 'year')
  
  mean = img.divide(10000).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = sn,
    scale = 1000,
    crs = 'EPSG:5070',
    bestEffort = True,
    maxPixels = 1e14,
    tileScale = 4
  )

  return(ee.Feature(None, mean)
    .set({
      'DOY': doy,
      'Year': year
    })
  )

# get modis NDVI collection
ndvi = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

# do the region reduction
snNdviCol = ndvi.map(reduceNdvi)

# Arrange the sample as a list of lists
snNdviDict = snNdviCol.reduceColumns(
  ee.Reducer.toList().repeat(3),
  ['Year', 'DOY', 'NDVI']
)

snNdviList = ee.List(snNdviDict.get('list'))
snNdviList = snNdviList.getInfo()

In [0]:
snNdviDf = pd.DataFrame(snNdviList)
snNdviDf = snNdviDf.transpose()
snNdviDf.columns = ['Year', 'DOY', 'NDVI']
snNdviDf = snNdviDf.astype({'Year': int, 'DOY': int, 'NDVI': float})

In [17]:
snNdviDf.head(10).style.hide_index()

Year,DOY,NDVI
2000,48,0.237385
2000,64,0.291306
2000,80,0.334815
2000,96,0.368556
2000,112,0.404176
2000,128,0.449644
2000,144,0.471491
2000,160,0.504714
2000,176,0.512157
2000,192,0.506266


Plot DOY chart

In [107]:
alt.Chart(snNdviDf).mark_line().encode(
  alt.X('DOY:O'),
  alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.1, 0.7))),
  alt.Color('Year:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Year:O', title='Year'),  
    alt.Tooltip('DOY:O', title='DOY'),
    alt.Tooltip('NDVI:Q', title='NDVI')
  ]
).interactive().properties(width=600)

alt.Chart(...)

In [108]:
line = alt.Chart(snNdviDf).mark_line().encode(
  x='DOY:O',
  y='median(NDVI):Q',
).interactive()

band = alt.Chart(snNdviDf).mark_errorband(extent='iqr').encode(
  x='DOY:O',
  y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.1, 0.7))),
).interactive().properties(width=600)

band + line

alt.LayerChart(...)

In [42]:
snNdviDfSub = snNdviDf[(snNdviDf['DOY'] >= 224) & (snNdviDf['DOY'] <= 272)]
snNdviDfSub = snNdviDfSub.groupby('Year').agg('min')

snPdsiDfSub = snPdsiDf[(snPdsiDf['DOY'] >= 1) & (snPdsiDf['DOY'] <= 272)]
snPdsiDfSub = snPdsiDfSub.groupby('Year').agg('mean')

test = pd.merge(snNdviDfSub, snPdsiDfSub, how='left', on='Year') \
  .drop(columns=['DOY_x', 'DOY_y', 'Month']) \
  .reset_index()

print(test.head(10))

   Year      NDVI      PDSI
0  2000  0.503463 -0.889241
1  2001  0.493344 -1.542462
2  2002  0.495781 -2.066630
3  2003  0.515792 -0.074927
4  2004  0.494522 -1.936395
5  2005  0.521287  1.717228
6  2006  0.513179  2.066451
7  2007  0.500129 -2.443901
8  2008  0.496119 -2.745732
9  2009  0.508540 -2.241880


In [111]:
test['Fit'] = np.poly1d(np.polyfit(test['PDSI'], test['NDVI'], 1))(test['PDSI'])
print(test.head(10))

   Year      NDVI      PDSI       Fit
0  2000  0.503463 -0.889241  0.503898
1  2001  0.493344 -1.542462  0.500240
2  2002  0.495781 -2.066630  0.497305
3  2003  0.515792 -0.074927  0.508459
4  2004  0.494522 -1.936395  0.498034
5  2005  0.521287  1.717228  0.518495
6  2006  0.513179  2.066451  0.520451
7  2007  0.500129 -2.443901  0.495192
8  2008  0.496119 -2.745732  0.493501
9  2009  0.508540 -2.241880  0.496323


In [112]:
points = alt.Chart(test).mark_circle(size=60).encode(
  x=alt.X('PDSI:Q', scale=alt.Scale(domain=(-5, 5))),
  y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.4, 0.6))),
  color=alt.Color('Year:O', scale=alt.Scale(scheme="magma")),
  tooltip=['Year', 'PDSI', 'NDVI']
).interactive()

fit = alt.Chart(test).mark_line().encode(
  x=alt.X('PDSI:Q', scale=alt.Scale(domain=(-5, 5))),
  y=alt.Y('Fit:Q', scale=alt.Scale(domain=(0.4, 0.6))),
  color=alt.value('#808080')
).properties(width=600)

fit + points 

alt.LayerChart(...)

# Landsat Image Change

In [0]:
#coefficients = {
#  'itcps': ee.Image.constant([0.0003, 0.0088, 0.0061, 0.0412, 0.0254, 0.0172]).multiply(10000),
#  'slopes': ee.Image.constant([0.8474, 0.8483, 0.9047, 0.8462, 0.8937, 0.9071])
#}

coefficients = {
  'itcps': ee.Image.constant([-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029]).multiply(10000),
  'slopes': ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])
}



# Define function to get and rename bands of interest from OLI.
def renameOLI(img):
  return(img.select(
    ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']),
		ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])
	))

# Define function to get and rename bands of interest from ETM+.
def renameETM(img):
  return(img.select(
		ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']),
		ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])
  ))

# Define function to apply harmonization transformation.
def etm2oli(img):
  return(img.select(ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])) \
    .multiply(coefficients['slopes']) \
    .add(coefficients['itcps']) \
    .round() \
    .toShort() \
    .addBands(img.select('pixel_qa')
  ))

# Define function to mask out clouds and cloud shadows.
def fmask(img):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5
  qa = img.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return(img.updateMask(mask))

# Define function to calculate NBR.
def calcNBR(img):
  return(img.normalizedDifference(ee.List(['NIR', 'SWIR2'])).rename('NBR'))

# Define function to prepare OLI images.
def prepOLI(img):
  orig = img
  img = renameOLI(img)
  img = fmask(img)
  img = calcNBR(img)
  return(ee.Image(img.copyProperties(orig, orig.propertyNames())))

# Define function to prepare ETM+ images.
def prepETM(img):
  orig = img
  img = renameETM(img)
  img = fmask(img)
  #img = etm2oli(img)
  img = calcNBR(img)
  return(ee.Image(img.copyProperties(orig, orig.propertyNames())))

In [79]:


#col = col.map(lambda img: img.set('Year', img.date().get('year')))

"""
# Subset collection to a set of distinct year representatives.
distinctYearCol = col.distinct('year')

# Define a filter that identifies images from the complete
# collection that match the 'year' from the distinct year collection
yearFilter = ee.Filter.equals(leftField='year', rightField='year')

# Define a join.
join = ee.Join.saveAll('year_matches')

# Apply the join and convert the resulting FeatureCollection to an ImageCollection.
joinCol = ee.ImageCollection(join.apply(distinctYearCol, col, yearFilter))
#print(joinCol.first().propertyNames().getInfo())


# Reduce intra-annaul collections by mean then reduce by region.
# Note that the date of the image is being hardcoded.
def reduceCol(img):
  yearCol = ee.ImageCollection.fromImages(img.get('year_matches'))
  yearMed = yearCol.reduce(ee.Reducer.mean()).rename(['NBR'])
  return(yearMed.set('year', img.get('year')))

# Apply median reduction among matching year collections.
# Blue River
medianComp = joinCol.map(reduceCol)
"""

"\n# Subset collection to a set of distinct year representatives.\ndistinctYearCol = col.distinct('year')\n\n# Define a filter that identifies images from the complete\n# collection that match the 'year' from the distinct year collection\nyearFilter = ee.Filter.equals(leftField='year', rightField='year')\n\n# Define a join.\njoin = ee.Join.saveAll('year_matches')\n\n# Apply the join and convert the resulting FeatureCollection to an ImageCollection.\njoinCol = ee.ImageCollection(join.apply(distinctYearCol, col, yearFilter))\n#print(joinCol.first().propertyNames().getInfo())\n\n\n# Reduce intra-annaul collections by mean then reduce by region.\n# Note that the date of the image is being hardcoded.\ndef reduceCol(img):\n  yearCol = ee.ImageCollection.fromImages(img.get('year_matches'))\n  yearMed = yearCol.reduce(ee.Reducer.mean()).rename(['NBR'])\n  return(yearMed.set('year', img.get('year')))\n\n# Apply median reduction among matching year collections.\n# Blue River\nmedianComp = joinCo

In [71]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
visParams = {'min':0, 'max':7000}

# Get the centroid of the area of interest to center the map.
#loc = sn.first().geometry().centroid().coordinates().reverse().getInfo()

# Create a folium map object.
myMap = folium.Map(
  location=loc,
  #tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
  #attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
  tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
  attr='Google',
  zoom_start=6,
  height=500)

# Add the RGB img to the map object.
#myMap.add_ee_layer(deltaRgb, visParams, 'RGB Delta')

# Add a layer control panel to the map.
#myMap.add_child(folium.LayerControl())
folium.LatLngPopup().add_to(myMap)
# Display the map.
display(myMap)

In [0]:
Latitude = 35.9665
Longitude = -118.6407

point = ee.Geometry.Point([Longitude, Latitude])

# Start and end dates
startDay = 176
endDay = 240

# get data
tmCol = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
etmCol = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
oliCol = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter collections and prepare them for merging.
oliCol = oliCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepOLI)
etmCol = etmCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepETM)
tmCol = tmCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepETM)

# Merge the collections.
col = oliCol \
  .merge(etmCol) \
  .merge(tmCol)

def reduceNbr(img):
  eeDate = img.date()
  year = eeDate.get('year')
  date = eeDate.format('YYYY-MM-dd')
  
  first = img.reduceRegion(
    reducer = ee.Reducer.first(),
    geometry = point,
    scale = 30,
    crs = 'EPSG:5070'
  )

  return(ee.Feature(None, first)
    .set({
      'SATELLITE': img.get('SATELLITE'),
      'Year': year,
      'Date': date
    })
  )
  
pointNbrCol = col.map(reduceNbr).filter(ee.Filter.notNull(['NBR']))

# Arrange the sample as a list of lists
pointNbrDict = pointNbrCol.reduceColumns(
  ee.Reducer.toList().repeat(4),
  ['Year', 'Date', 'NBR', 'SATELLITE']
)

pointNbrList = ee.List(pointNbrDict.get('list'))
pointNbrList = pointNbrList.getInfo()


In [119]:
print(pointNbrList)

[[2013, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017, 2017, 2018, 2018, 2018, 2018, 2019, 2019, 2019, 2019, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2017, 2017, 2017, 2018, 2018, 2018, 2018, 2019, 2019, 2019, 1999, 1999, 1999, 1999, 2000, 2000, 2000, 2000, 2001, 2001, 2001, 2002, 2002, 2002, 2003, 2004, 2004, 2005, 2005, 2005, 2006, 2006, 2006, 2007, 2008, 2008, 2008, 2008, 2008, 2009, 2009, 2009, 2010, 2010, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2016, 2016, 2016, 2016, 2017, 2018, 2018, 2018, 2019, 1999, 1999, 2000, 2000, 2000, 2000, 2001, 2001, 2001, 2002, 2002, 2002, 2002, 2003, 2004, 2004, 2004, 2005, 2006, 2006, 2008, 2009, 2009, 2009, 2009, 2010, 2010, 2010, 2010, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2013, 2013, 2013, 2014, 2014, 2015, 2015, 2016, 2016, 2018, 2019, 2019, 1984, 1985, 1986, 1986, 1986, 1987, 19

In [133]:
pointNbrDf = pd.DataFrame(pointNbrList)
pointNbrDf = pointNbrDf.transpose()
pointNbrDf.columns = ['Year', 'Date', 'NBR', 'Satellite']
pointNbrDf = pointNbrDf.astype({'Year': int, 'Date': str, 'NBR': float, 'Satellite': str})
pointNbrDf#.head(10).style.hide_index()

,Year,Date,NBR,Satellite
0,2013,2013-06-30,0.638326,LANDSAT_8
1,2013,2013-07-16,0.616417,LANDSAT_8
2,2013,2013-08-01,0.655117,LANDSAT_8
3,2013,2013-08-17,0.618759,LANDSAT_8
4,2014,2014-07-03,0.596919,LANDSAT_8
5,2014,2014-08-04,0.590498,LANDSAT_8
6,2014,2014-08-20,0.600000,LANDSAT_8
7,2015,2015-07-06,0.590945,LANDSAT_8
8,2015,2015-07-22,0.602592,LANDSAT_8
9,2015,2015-08-07,0.548442,LANDSAT_8


In [130]:
nbrSeries = alt.Chart(pointNbrDf).mark_circle(size=60).encode(
  x=alt.X('Date:T'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('NBR:Q'), #, scale=alt.Scale(domain=(0.4, 0.6))
  color=alt.Color('Satellite:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('NBR:Q', title='NBR'),
    alt.Tooltip('Satellite:O', title='Satellite') 
  ]
).interactive().properties(width=600)

nbrSeries

alt.Chart(...)

In [134]:
nbrSeries = alt.Chart(pointNbrDf).mark_circle(size=60).encode(
  x=alt.X('Date:T'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('NBR:Q'), #, scale=alt.Scale(domain=(0.4, 0.6))
  color=alt.Color('Satellite:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('NBR:Q', title='NBR'),
    alt.Tooltip('Satellite:O', title='Satellite') 
  ]
).interactive().properties(width=600)

nbrSeries

alt.Chart(...)

In [145]:
line = alt.Chart(snNdviDf).mark_line().encode(
  x='DOY:O',
  y='median(NDVI):Q',
).interactive()


line = alt.Chart(pointNbrDf).mark_line().encode(
  x=alt.X('Year:O'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('median(NBR):Q') #, scale=alt.Scale(domain=(0.4, 0.6))
).interactive().properties(width=600)

band = alt.Chart(pointNbrDf).mark_errorband(extent='iqr').encode(
  x=alt.X('Year:O'),
  y=alt.Y('NBR:Q'),
).interactive().properties(width=600)

band + line

alt.LayerChart(...)

# NEX-DCP

In [169]:
dcp = ee.ImageCollection("NASA/NEX-DCP30_ENSEMBLE_STATS") \
  .select(['tasmax_median','tasmin_median', 'pr_median']) \
  .filter(ee.Filter.And(
    ee.Filter.eq('scenario', 'rcp85'),
    ee.Filter.date('2020-01-01', '2070-12-31')
  ))

def reduceDcp(img):
  eeDate = img.date()
  year = eeDate.get('year')
  date = eeDate.format('YYYY-MM-dd')
  
  first = img.reduceRegion(
    reducer = ee.Reducer.first(),
    geometry = point,
    scale = 5000,
    crs = 'EPSG:5070'
  )

  return(ee.Feature(None, first)
    .set({
      'Scenario': img.get('scenario'),
      'Year': year,
      'Date': date
    })
  )
  
pointDcpCol = dcp.map(reduceDcp).filter(ee.Filter.notNull(['tasmax_median','tasmin_median', 'pr_median']))


# Arrange the sample as a list of lists
pointDcpDict = pointDcpCol.reduceColumns(
  ee.Reducer.toList().repeat(6),
  ['Year', 'Date', 'Scenario', 'tasmax_median', 'tasmin_median', 'pr_median']
)

pointDcpList = ee.List(pointDcpDict.get('list'))
pointDcpList = pointDcpList.getInfo()
print(pointDcpList)

[[2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2026, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2027, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2029, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2030, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2031, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2032, 2033, 2033, 2033, 2033, 2033, 2033, 2033, 2033, 2033, 2033, 20

In [171]:
pointDcpDf = pd.DataFrame(pointDcpList)
pointDcpDf = pointDcpDf.transpose()
pointDcpDf.columns = ['Year', 'Date', 'Scenario', 'Temp-max', 'Temp-min', 'Precip-rate']
pointDcpDf = pointDcpDf.astype({'Year': int, 'Date': str, 'Scenario': str, 'Temp-max': float, 'Temp-min': float, 'Precip-rate': float})
pointDcpDf.head(10).style.hide_index()

Year,Date,Scenario,Temp-max,Temp-min,Precip-rate
2020,2020-01-01,rcp85,280.366,270.82,3.96475e-05
2020,2020-02-01,rcp85,283.064,271.683,4.72716e-05
2020,2020-03-01,rcp85,285.298,272.548,3.22458e-05
2020,2020-04-01,rcp85,289.241,275.644,1.45603e-05
2020,2020-05-01,rcp85,292.1,278.564,6.63788e-06
2020,2020-06-01,rcp85,297.41,283.433,1.65511e-06
2020,2020-07-01,rcp85,301.529,286.521,1.19578e-06
2020,2020-08-01,rcp85,300.827,286.216,6.39879e-07
2020,2020-09-01,rcp85,298.103,282.839,1.13377e-06
2020,2020-10-01,rcp85,292.626,278.543,7.26352e-06


In [173]:
pointDcpDf['Precip-mm'] = pointDcpDf['Precip-rate']*86400*30
pointDcpDf.head(10).style.hide_index()

Year,Date,Scenario,Temp-max,Temp-min,Precip-rate,Precip-mm
2020,2020-01-01,rcp85,280.366,270.82,3.96475e-05,102.766
2020,2020-02-01,rcp85,283.064,271.683,4.72716e-05,122.528
2020,2020-03-01,rcp85,285.298,272.548,3.22458e-05,83.5811
2020,2020-04-01,rcp85,289.241,275.644,1.45603e-05,37.7403
2020,2020-05-01,rcp85,292.1,278.564,6.63788e-06,17.2054
2020,2020-06-01,rcp85,297.41,283.433,1.65511e-06,4.29005
2020,2020-07-01,rcp85,301.529,286.521,1.19578e-06,3.09945
2020,2020-08-01,rcp85,300.827,286.216,6.39879e-07,1.65857
2020,2020-09-01,rcp85,298.103,282.839,1.13377e-06,2.93874
2020,2020-10-01,rcp85,292.626,278.543,7.26352e-06,18.827


In [174]:
line = alt.Chart(pointDcpDf).mark_line().encode(
  x='Year:O',
  y='sum(Precip-mm):Q',
).interactive()

line

alt.Chart(...)

RCP: https://en.wikipedia.org/wiki/Representative_Concentration_Pathway

In [176]:
line = alt.Chart(pointDcpDf).mark_line().encode(
  x='Year:O',
  y='mean(Temp-max):Q',
).interactive()

line

alt.Chart(...)